In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import gzip
import zipfile
import shutil
from time import sleep
import pickle
from time import sleep
import sys
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
import tensorflow as tf
import tensorflow_hub as hub
!pip install tensorflow_text
import tensorflow_text

Mounted at /content/drive
     |████████████████████████████████| 4.9 MB 5.1 MB/s 


In [ ]:
df_marks = pd.read_csv('/content/drive/MyDrive/train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('/content/drive/MyDrive/sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [ ]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [ ]:
with open('/content/drive/MyDrive/id_querry_spelled.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)
with open('/content/drive/MyDrive/title_data.pickle', 'rb') as file:
    title_data = pickle.load(file)

In [ ]:
model1 = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

In [ ]:
docs = all_groups['DocumentId'].values
q_ids = all_groups['QueryId'].values

In [ ]:
neuro_feats = dict()
neuro_feats['use-multilingual-qa/3_unnormed_512_texts'] = []

In [ ]:
i = -1
with gzip.open("/content/drive/MyDrive/preembeddings.gz") as file:
    for line in tqdm(file):
        a = line.decode('utf-8', errors = 'ignore').lower().strip('\n').split('\t')
        a = ' '.join(a[0].split()[:512])
        i+=1
        b = title_data[docs[i]]
        cur_q = id_querry_clean[q_ids[i]]
        Q_emb = model1.signatures['question_encoder'](tf.constant(cur_q))
        D_emb = model1.signatures['response_encoder'](input=tf.constant(b), context=tf.constant(a))
        neuro_feats['use-multilingual-qa/3_unnormed_512_texts'].append(cosine_similarity(Q_emb['outputs'].numpy(), D_emb['outputs'].numpy()).flatten()[0])
with open('/content/drive/MyDrive/neuro_feats_unnormed_512_texts-multilang-qa-3_upd.pickle', 'wb') as file:
    pickle.dump(neuro_feats, file)

606050it [7:05:28, 23.74it/s]


In [ ]:
a

''

In [ ]:
i

-1